In [1]:
import json
from pathlib import Path

import pandas as pd


def round_and_percentage(num: float) -> float:
    return round(num * 100, 2)


model_prefix_name_map = {"transk": "Transkribus", "tess": "Tesseract", "trocr": "TrOCR"}
dataset_map = {
    "ub_smi": "GT-Sámi (without base)",
    "smi": "GT-Sámi",
    "smi_nor": "GT-Sámi + GT-Nor",
    "smi_pred": "GT-Sámi + Pred-Sámi",
    "smi_nor_pred": "GT-Sámi + GT-Nor + Pred-Sámi",
}

In [2]:
index = pd.MultiIndex.from_product(
    [["CER", "WER", "Sámi letter F1"], ["overall", "sma", "sme", "smj", "smn"]]
)

In [3]:
from collections import defaultdict

## TODO: bytt ut med testset data
best_tesseract_model = "../output/evaluation/line_level/tess_smi_nor_pred"
best_trocr_model = "../output/evaluation/line_level/trocr_smi_nor"
best_transkribus_model = "../output/evaluation/line_level/transk_smi_nor_pred"


langs_in_order = ["all", "sma", "sme", "smj", "smn"]
scores_in_order = ["CER", "WER", "Sámi letter F1"]
index = pd.MultiIndex.from_product([scores_in_order, langs_in_order])

df_data = {}

for model_p in [best_trocr_model, best_tesseract_model, best_transkribus_model]:
    model_p = Path(model_p)
    model_prefix, _, model_info = model_p.name.partition("_")
    model_scores = defaultdict(defaultdict)

    for lang_file in model_p.glob("*rows.json"):
        lang = lang_file.stem.split("_")[0]
        scores = json.loads(lang_file.read_text())
        model_scores["WER"][lang] = round_and_percentage(scores["WER_concat"])
        model_scores["CER"][lang] = round_and_percentage(scores["CER_concat"])
        model_scores["Sámi letter F1"][lang] = round_and_percentage(scores["special_char_F1_mean"])

    df_data[model_prefix_name_map[model_prefix]] = [
        model_scores[score][lang] for score in scores_in_order for lang in langs_in_order
    ]

In [4]:
df = pd.DataFrame(df_data, index=index)
df

TrOCR  Tesseract  Transkribus
CER            all    1.95       4.40         1.19
               sma    1.10       2.67         0.35
               sme    1.93       4.58         1.32
               smj    1.01       4.19         0.29
               smn    2.70       3.31         0.42
WER            all    8.88       7.96         4.47
               sma    7.69       3.85         2.66
               sme    8.50       8.03         4.84
               smj    4.85       6.80         1.46
               smn   14.19       9.14         2.23
Sámi letter F1 all   94.55      94.76        97.32
               sma   92.67      95.40       100.00
               sme   94.47      94.68        97.14
               smj  100.00      97.44        98.72
               smn   94.81      94.91        98.65

In [5]:
print(
    df.to_latex(
        float_format="%.2f",
    )
)

\begin{tabular}{llrrr}
\toprule
 &  & TrOCR & Tesseract & Transkribus \\
\midrule
\multirow[t]{5}{*}{CER} & all & 1.95 & 4.40 & 1.19 \\
 & sma & 1.10 & 2.67 & 0.35 \\
 & sme & 1.93 & 4.58 & 1.32 \\
 & smj & 1.01 & 4.19 & 0.29 \\
 & smn & 2.70 & 3.31 & 0.42 \\
\cline{1-5}
\multirow[t]{5}{*}{WER} & all & 8.88 & 7.96 & 4.47 \\
 & sma & 7.69 & 3.85 & 2.66 \\
 & sme & 8.50 & 8.03 & 4.84 \\
 & smj & 4.85 & 6.80 & 1.46 \\
 & smn & 14.19 & 9.14 & 2.23 \\
\cline{1-5}
\multirow[t]{5}{*}{Sámi letter F1} & all & 94.55 & 94.76 & 97.32 \\
 & sma & 92.67 & 95.40 & 100.00 \\
 & sme & 94.47 & 94.68 & 97.14 \\
 & smj & 100.00 & 97.44 & 98.72 \\
 & smn & 94.81 & 94.91 & 98.65 \\
\cline{1-5}
\bottomrule
\end{tabular}

